# Creds

> Fill in a module description here

In [1]:
#| default_exp google.creds

In [2]:
#| hide
from nbdev.showdoc import *


In [3]:
#| exporti

import os
import json

from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
from google.auth.exceptions import RefreshError
from google_auth_oauthlib.flow import InstalledAppFlow

from dotenv import load_dotenv, set_key

In [4]:
#|export
SCOPE = [
        "https://www.googleapis.com/auth/drive",
        "https://www.googleapis.com/auth/drive.file",
        "https://www.googleapis.com/auth/drive.metadata.readonly",
        "https://www.googleapis.com/auth/drive.readonly",
    ]

def get_creds_from_file(scope, token_file_path="env_token.json", credentials_file_path="env_creds.json"):

    """
    generate creds here, https://console.cloud.google.com/apis/credentials/oauthclient
    for a new implementation you'll need to
    1. set up a project at console.cloud.google.com,
    2. enable (turn on) the appropriate APIs for that project
    3. configure OAuth access (your python script will use an Oauth token to authenticate, so your project needs the OAuth screen)
    4. download the creds file and store it as env_creds.json
    """

    # If modifying these scopes, delete the file token.json.
    

    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if token_file_path and os.path.exists(token_file_path):
        creds = Credentials.from_authorized_user_file(token_file_path, scope)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            # if creds token can be refreshed, do it.
            creds.refresh(Request())
        else:
            # If there are no (valid) credentials available, run the Oauth flow
            flow = InstalledAppFlow.from_client_secrets_file(
                credentials_file_path, scope
            )
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open(token_file_path, "w") as token:
            token.write(creds.to_json())
    return creds

In [5]:
#|export

def get_creds_from_env(scope, token_env_key : str = 'GDOC_TOKEN', credentials_env_key : str = 'GDOC_CLIENT', env_file = '.env'):
    """
    credentials should be stored in the .env file as a json (NOT A STRING)
    token will be stored as a string (NOT A JSON OBJ)
    
    generate creds here, https://console.cloud.google.com/apis/credentials/oauthclient
    for a new implementation you'll need to
    1. set up a project at console.cloud.google.com,
    2. enable (turn on) the appropriate APIs for that project
    3. configure OAuth access (your python script will use an Oauth token to authenticate, so your project needs the OAuth screen)
    4. download the creds file and store it as env_creds.json
    """

    creds = None

    load_dotenv(env_file)
    token_str = os.environ.get(token_env_key)
    token_json = json.loads(token_str) if token_str else None

    if token_json:
        creds = Credentials.from_authorized_user_info(token_json, scope)
    
    if creds and creds.expired and creds.refresh_token:
        # if creds token can be refreshed, do it.
        try:
            creds.refresh(Request())
            
        except RefreshError as e:
            print(e)

    if not creds or not creds.valid:
        # If there are no (valid) credentials available, run the Oauth flow
        credentials_json = json.loads(os.environ.get(credentials_env_key))

        print(credentials_env_key,credentials_json)

        flow = InstalledAppFlow.from_client_config(
            credentials_json, scope
        )
        creds = flow.run_local_server(port=0)

    # Save the credentials for the next run
    set_key(env_file, token_env_key, creds.to_json())

    return creds


In [6]:
creds = get_creds_from_env(credentials_env_key= 'GDOC_CLIENT', token_env_key = 'GDOC_TOKEN',  scope = SCOPE, env_file = '../.env')

#### sample implementation of get_creds_from_env

In [7]:
#| hide
import nbdev

nbdev.nbdev_export()